In [1]:
import torch
import open3d as o3d
import numpy as np  


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:

with open('./pointdata.pth', 'rb') as file: 
    loaded_data = torch.load(file)
    point = loaded_data
    
    print(loaded_data.keys())
    print(type(loaded_data['coord'][0][0]))
    # print(type(loaded_data['normal'][0][0]))
    # loaded_data['labels'] = loaded_data['labels'].astype(int) 
    # print(type(loaded_data['labels'][0]))
    print(torch.min(loaded_data['coord']))
    
    # mask = np.full(loaded_data['coord'].shape[0], False, dtype=bool)  
  
    # 计算需要设置为 True 的元素数量  
    # num_true = mask.size // 10  # 十分之一的元素数量  
    
    # 随机选择位置并将其设为 True  
    # indices = np.random.choice(mask.size, num_true, replace=False)  
    # mask = mask.flatten()  
    # mask[indices] = True  
    # mask = mask.reshape(loaded_data['coord'].shape[0]) 
    # print(mask.shape,mask)
    
    mask = (loaded_data['segment'] != 0).cpu().numpy()
    feat = loaded_data['feat'].cpu().detach().numpy()
    print('feat',feat.shape,feat)
    coord = loaded_data['coord'].cpu().numpy()
    print(coord)
    offset = loaded_data['offset_vector'].cpu().detach().numpy()
    print(offset)
    # points = coord
    # points[mask] = coord[mask]+offset[mask]
    points = coord+offset
    print(points)
    point_cloud = o3d.geometry.PointCloud()  
    point_cloud.points = o3d.utility.Vector3dVector(points)  

dict_keys(['coord', 'grid_coord', 'segment', 'offset_vector', 'offset', 'feat', 'batch', 'serialized_depth', 'serialized_code', 'serialized_order', 'serialized_inverse', 'sparse_shape', 'sparse_conv_feat', 'pad', 'unpad', 'cu_seqlens_key'])
<class 'torch.Tensor'>
tensor(-1.3277, device='cuda:0')
feat (2520, 64) [[ 1.8150625  -0.21156877 -0.6854651  ... -0.9145881   3.6848195
  -0.67851233]
 [ 2.6054084  -0.27915195 -0.48477268 ... -0.09085202  3.699931
  -0.71023357]
 [ 4.3451834  -2.056975    1.7662511  ... -1.3247106   6.4151497
  -1.5648804 ]
 ...
 [ 1.7546022   0.96389043  0.9208914  ... -0.02010047  0.846984
  -1.8479385 ]
 [ 1.1326145  -0.501377    1.7411212  ...  0.10290253 -0.4396162
  -1.7515671 ]
 [ 1.7888358   0.07827127  1.4333152  ... -0.5393679  -0.23504615
  -1.3325466 ]]
[[ 0.23419605 -0.86805135 -0.59151775]
 [ 0.24591915 -0.79034305 -0.09641358]
 [ 0.21403874 -0.59407777  0.2598418 ]
 ...
 [ 0.2530775   0.7718995  -0.36538458]
 [ 0.29401678  0.721824   -0.25102797]
 [

In [ ]:
for key in point.keys():
    if key != 'serialized_depth':
        print(key,point[key].shape,point[key])
    

coord torch.Size([1960, 3]) tensor([[ 0.4257, -0.5839, -0.6326],
        [ 0.4318, -0.5143, -0.2401],
        [ 0.4347, -0.5247, -0.2558],
        ...,
        [ 0.4775,  0.8250, -0.8549],
        [ 0.4954,  0.9436, -0.0997],
        [ 0.4552,  0.9401, -0.1067]], device='cuda:0')
grid_coord torch.Size([1960, 3]) tensor([[31,  4,  5],
        [31,  5, 13],
        [31,  5, 12],
        ...,
        [32, 32,  0],
        [32, 34, 16],
        [32, 34, 15]], device='cuda:0')
segment torch.Size([1960]) tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
offset_vector torch.Size([1960, 3]) tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        ...,
        [ 0.9993, -1.7156,  0.2347],
        [ 0.9993, -1.7156,  0.2347],
        [ 0.9993, -1.7156,  0.2347]], device='cuda:0')
offset torch.Size([1]) tensor([1960], device='cuda:0')
feat torch.Size([1960, 64]) tensor([[ 1.5956, -0.8234, -0.1293,  ...,  0.0142,  2.0401, -0.6389],
    

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
import torch.nn as nn

In [ ]:
# 计算dir loss
offset_logits = point['offset_vector']
offset_target = offset_logits.clone()
offset_target[:,0] = offset_logits[:,0] + 0.1

print(offset_logits)
print(offset_target)
offset_pred_norm = torch.norm(offset_logits,dim=1).view(-1,1)
offset_pred_dir = torch.div(offset_logits, offset_pred_norm)
mask1 = offset_pred_norm > 0.01
print(offset_pred_dir)
offset_target_norm = torch.norm(offset_target, dim=1).view(-1,1)
offset_target_dir = torch.div(offset_target, offset_target_norm)
mask2 = offset_target_norm > 0.01
mask = mask1 & mask2
mask = mask.squeeze()

print(offset_target_dir)
print(mask)
dir_mat = torch.sum(offset_pred_dir * offset_target_dir, dim=1)
dir_mat = dir_mat - 1
dir_mat = dir_mat * dir_mat
loss_dir = torch.div(torch.sum(dir_mat[mask]),mask.shape[0])
print('loss dir',loss_dir)

cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6) 
loss = 1 - cos_sim(offset_logits, offset_target)
print(loss.mean())

loss = nn.MSELoss()(offset_logits,offset_target)
print(loss)


tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-0.0055,  0.0585, -0.0158],
        ...,
        [-0.8421, -0.1897, -1.3782],
        [-0.8421, -0.1897, -1.3782],
        [-0.8421, -0.1897, -1.3782]], device='cuda:0')
tensor([[ 0.1000,  0.0000,  0.0000],
        [ 0.1000,  0.0000,  0.0000],
        [ 0.0945,  0.0585, -0.0158],
        ...,
        [-0.7421, -0.1897, -1.3782],
        [-0.7421, -0.1897, -1.3782],
        [-0.7421, -0.1897, -1.3782]], device='cuda:0')
tensor([[    nan,     nan,     nan],
        [    nan,     nan,     nan],
        [-0.0897,  0.9616, -0.2595],
        ...,
        [-0.5178, -0.1166, -0.8475],
        [-0.5178, -0.1166, -0.8475],
        [-0.5178, -0.1166, -0.8475]], device='cuda:0')
tensor([[ 1.0000,  0.0000,  0.0000],
        [ 1.0000,  0.0000,  0.0000],
        [ 0.8420,  0.5208, -0.1406],
        ...,
        [-0.4707, -0.1203, -0.8741],
        [-0.4707, -0.1203, -0.8741],
        [-0.4707, -0.1203, -0.8741]], devic

In [7]:
with open('./testdata.pth', 'rb') as file: 
    loaded_data = torch.load(file)
    print(loaded_data.keys())
    print(loaded_data['grid_coord'])
    print(loaded_data['offset'])

RuntimeError: PytorchStreamReader failed reading file data/0: invalid header or archive is corrupted